# Оценка качества моделей классификации.

Будем тренироваться интерпретировать качество моделей классификации.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

%pylab inline

### Задача: датасет о кредитном скоринге. 
* Колонки 0, 1, ..., 13 - анонимизированные характеристики клиентов (некоторые числовые, некоторые категориальные). 
* Target - целевая переменная: 1 - выдать кредит, 0 - не выдать кредит.

На этих данных было обучено три различных алгоритма классификации и получены предсказания:
* pred1_probs - предсказанные вероятности положительного класса, полученные алгоритмом 1.
* pred1_classes - предсказанные алгоритмом 1 классы
* pred2_probs, pred2_classes, pred3_probs, pred3_classes - аналогичные величины для алгоритмов 2 и 3

Наша задача - оценить качество каждого из трех алгоритмов и разобраться, какой из алгоритмов лучше всего работает в данной задаче.

In [ ]:
df = pd.read_csv("credit_scoring_example1.csv")
df.head()

In [ ]:
df.shape

Посмотрим, сбалансированная ли выборка.

In [ ]:
len(df[df.target == 0]), len(df[df.target == 1])

Оценим **accuracy** - долю правильных ответов каждого из алгоритмов.

In [ ]:
from sklearn.metrics import accuracy_score

print('алгоритм 1:', accuracy_score(df['target'], df['pred1_classes']))
print('алгоритм 2:', accuracy_score(df['target'], df['pred2_classes']))
print('алгоритм 3:', accuracy_score(df['target'], df['pred3_classes']))

***С точки зрения метрики accuracy второй алгоритм работает немного лучше остальных***. Посмотрим на другие метрики.

Теперь посмотрим на **precision** и **recall**.

In [ ]:
from sklearn.metrics import precision_score, recall_score

print('алгоритм 1:')
print('точность:', precision_score(df['target'], df['pred1_classes']))
print('полнота:',  recall_score(df['target'], df['pred1_classes']))
print('алгоритм 2:')
print('точность:', precision_score(df['target'], df['pred2_classes']))
print('полнота:',  recall_score(df['target'], df['pred2_classes']))
print('алгоритм 3:')
print('точность:', precision_score(df['target'], df['pred3_classes']))
print('полнота:',  recall_score(df['target'], df['pred3_classes']))

Самым полезным инструментом в практических задачах является ***матрица ошибок***. Смотря на неё и регулируя порог, определяющий классы, мы можем достичь оптимального решения нашей задачи.

In [ ]:
import itertools
from sklearn.metrics import confusion_matrix

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
 
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)
 
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")
 
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
print('алгоритм 1')
plot_confusion_matrix(confusion_matrix(df['target'], df['pred1_classes']), classes=['0','1'],
                        title='Confusion matrix, without normalization')
show()
print('алгоритм 2')
plot_confusion_matrix(confusion_matrix(df['target'], df['pred2_classes']), classes=['0','1'],
                        title='Confusion matrix, without normalization')
show()
print('алгоритм 3')
plot_confusion_matrix(confusion_matrix(df['target'], df['pred3_classes']), classes=['0','1'],
                        title='Confusion matrix, without normalization')
show()

Давайте определимся с целями нашего предсказания:
    
* **Вариант 1**: хотим выдать кредит максимальному числу людей среди тех, кто мог бы его вернуть. При этом не хотим много ошибаться (общее число ошибок не более 35%).
  
1) Мы не хотим, чтобы алгоритм много ошибался: accuracy $\geq$ 0.65.

2) Кроме того полнота должна быть как можно больше (число в левом нижнем квадрате матрицы ошибок минимально).

Будем изменять порог вероятности в цикле и смотреть на результаты.

Смотрим на **модель 1**:

In [ ]:
for t in np.arange(0.1,1,0.1):
    pred1_thr = [1 if x >= t else 0 for x in df['pred1_probs']]
    print('threshold =', t)
    print('полнота(recall):', recall_score(df['target'],pred1_thr))
    print('accuracy:', accuracy_score(df['target'],pred1_thr))
    print()

In [ ]:
for t in np.arange(0.35,0.6,0.025):
    pred1_thr = [1 if x >= t else 0 for x in df['pred1_probs']]
    print('threshold =', t)
    print('полнота(recall):', recall_score(df['target'],pred1_thr))
    print('accuracy:', accuracy_score(df['target'],pred1_thr))
    print()

***Наилучший результат:***

threshold = 0.375

полнота(recall): 0.71

accuracy: 0.65

In [ ]:
pred1_thr = [1 if x >= 0.375 else 0 for x in df['pred1_probs']]

plot_confusion_matrix(confusion_matrix(df['target'], pred1_thr), classes=['0','1'],
                        title='Confusion matrix, without normalization')

Теперь посмотрим на **модель 2**:

In [ ]:
for t in np.arange(0.1,1,0.1):
    pred2_thr = [1 if x >= t else 0 for x in df['pred2_probs']]
    print('threshold =', t)
    print('полнота(recall):', recall_score(df['target'],pred2_thr))
    print('accuracy:', accuracy_score(df['target'],pred2_thr))
    print()

In [ ]:
for t in np.arange(0.25,0.8,0.025):
    pred2_thr = [1 if x >= t else 0 for x in df['pred2_probs']]
    print('threshold =', t)
    print('полнота(recall):', recall_score(df['target'],pred2_thr))
    print('accuracy:', accuracy_score(df['target'],pred2_thr))
    print()

***Наилучший результат:***

threshold = 0.3

полнота(recall): 0.837
    
accuracy: 0.675

In [ ]:
pred2_thr = [1 if x >= 0.3 else 0 for x in df['pred2_probs']]

plot_confusion_matrix(confusion_matrix(df['target'], pred2_thr), classes=['0','1'],
                        title='Confusion matrix, without normalization')

Посмотрим на **модель 3**:

In [ ]:
for t in np.arange(0.1,1,0.1):
    pred3_thr = [1 if x >= t else 0 for x in df['pred3_probs']]
    print('threshold =', t)
    print('полнота(recall):', recall_score(df['target'],pred3_thr))
    print('accuracy:', accuracy_score(df['target'],pred3_thr))
    print()

In [ ]:
for t in np.arange(0.35,0.6,0.025):
    pred3_thr = [1 if x >= t else 0 for x in df['pred3_probs']]
    print('threshold =', t)
    print('полнота(recall):', recall_score(df['target'],pred3_thr))
    print('accuracy:', accuracy_score(df['target'],pred3_thr))
    print()

***Наилучший результат:***

threshold = 0.4

полнота(recall): 0.414
    
accuracy: 0.646 (наибольшая точность)

***Модель не проходит по критериям (точность меньше 0.65)!***

In [ ]:
pred3_thr = [1 if x >= 0.4 else 0 for x in df['pred3_probs']]

plot_confusion_matrix(confusion_matrix(df['target'], pred3_thr), classes=['0','1'],
                        title='Confusion matrix, without normalization')

**Выводы:** с поставленной задачей лучше всего справляется модель 2 (полнота: 0.837,
accuracy: 0.675). 

Наша цель может звучать и по-другому:
    
* **Вариант 2**: среди тех, кому модель выдает кредит, должно быть меньше всего ошибок (мало людей, кто его на самом деле не вернет - на них будем терять деньги). 
    При этом не хотим много ошибаться (общее число ошибок не более 35%).
  
1) Мы не хотим, чтобы алгоритм много ошибался: accuracy $\geq$ 0.65.

2) Кроме того точность должна быть как можно больше (число в правом верхнем квадрате матрицы ошибок минимально).

Будем изменять порог вероятности в цикле и смотреть на результаты.

Смотрим на **модель 1**:

In [ ]:
for t in np.arange(0.1,1,0.1):
    pred1_thr = [1 if x >= t else 0 for x in df['pred1_probs']]
    print('threshold =', t)
    print('точность(precision):', precision_score(df['target'],pred1_thr))
    print('accuracy:', accuracy_score(df['target'],pred1_thr))
    print()

In [ ]:
for t in np.arange(0.35,0.6,0.1):
    pred1_thr = [1 if x >= t else 0 for x in df['pred1_probs']]
    print('threshold =', t)
    print('точность(precision):', precision_score(df['target'],pred1_thr))
    print('accuracy:', accuracy_score(df['target'],pred1_thr))
    print()

***Наилучший результат:***

threshold = 0.55

точность(precision): 0.733
    
accuracy: 0.653

In [ ]:
pred1_thr = [1 if x >= 0.55 else 0 for x in df['pred1_probs']]

plot_confusion_matrix(confusion_matrix(df['target'], pred1_thr), classes=['0','1'],
                        title='Confusion matrix, without normalization')

Смотрим на **модель 2**:

In [ ]:
for t in np.arange(0.1,1,0.1):
    pred2_thr = [1 if x >= t else 0 for x in df['pred2_probs']]
    print('threshold =', t)
    print('точность(precision):', precision_score(df['target'],pred2_thr))
    print('accuracy:', accuracy_score(df['target'],pred2_thr))
    print()

In [ ]:
for t in np.arange(0.25,0.8,0.025):
    pred2_thr = [1 if x >= t else 0 for x in df['pred2_probs']]
    print('threshold =', t)
    print('точность(precision):', precision_score(df['target'],pred2_thr))
    print('accuracy:', accuracy_score(df['target'],pred2_thr))
    print()

***Наилучший результат:***

threshold = 0.725

точность(precision): 0.837
    
accuracy: 0.65

In [ ]:
pred2_thr = [1 if x >= 0.725 else 0 for x in df['pred2_probs']]

plot_confusion_matrix(confusion_matrix(df['target'], pred2_thr), classes=['0','1'],
                        title='Confusion matrix, without normalization')

Смотрим на **модель 3**:

In [ ]:
for t in np.arange(0.1,1,0.1):
    pred3_thr = [1 if x >= t else 0 for x in df['pred3_probs']]
    print('threshold =', t)
    print('точность(precision):', precision_score(df['target'],pred3_thr))
    print('accuracy:', accuracy_score(df['target'],pred3_thr))
    print()

In [ ]:
for t in np.arange(0.4,0.65,0.025):
    pred3_thr = [1 if x >= t else 0 for x in df['pred3_probs']]
    print('threshold =', t)
    print('точность(precision):', precision_score(df['target'],pred3_thr))
    print('accuracy:', accuracy_score(df['target'],pred3_thr))
    print()

***Наилучший результат:***

threshold = 0.575

точность(precision): 0.714
    
accuracy: 0.642
    
***Модель не проходит по критериям (точность меньше 0.65)!***

In [ ]:
pred3_thr = [1 if x >= 0.575 else 0 for x in df['pred3_probs']]

plot_confusion_matrix(confusion_matrix(df['target'], pred3_thr), classes=['0','1'],
                        title='Confusion matrix, without normalization')

Также (**Вариант 3**) можно потребовать, чтобы и точность, и полнота были высокими, то есть обе ошибки в матрице ошибок были маленькие. Как вариант - можно минимизировать F1-score (среднее гармоническое между точностью и полнотой).

Найдем порог для достижения максимального F1-score для наилучшей модели (вторая модель) и построим матрицу ошибок (***проверьте, что F1-score у двух оставшихся моделей ниже!***).

In [ ]:
from sklearn.metrics import f1_score

for t in np.arange(0.1,1,0.1):
    pred1_thr = [1 if x >= t else 0 for x in df['pred1_probs']]
    print('threshold =', t)
    print('f1-score:', f1_score(df['target'],pred1_thr))
    print('accuracy:', accuracy_score(df['target'],pred1_thr))
    print()

In [ ]:
from sklearn.metrics import f1_score

for t in np.arange(0.35,0.6,0.025):
    pred1_thr = [1 if x >= t else 0 for x in df['pred1_probs']]
    print('threshold =', t)
    print('f1-score:', f1_score(df['target'],pred1_thr))
    print('accuracy:', accuracy_score(df['target'],pred1_thr))
    print()

***Наилучший результат:***

threshold = 0.375

f1-score: 0.644
    
accuracy: 0.65

In [ ]:
pred1_thr = [1 if x >= 0.375 else 0 for x in df['pred1_probs']]

plot_confusion_matrix(confusion_matrix(df['target'], pred1_thr), classes=['0','1'],
                        title='Confusion matrix, without normalization')

Посмотрим на roc-auc.

In [ ]:
from sklearn.metrics import roc_auc_score

print('алгоритм 1:', roc_auc_score(df['target'], df['pred1_probs']))
print('алгоритм 2:', roc_auc_score(df['target'], df['pred2_probs']))
print('алгоритм 3:', roc_auc_score(df['target'], df['pred3_probs']))

Построим roc-кривую для каждой модели и отметим на графике roc-auc.

In [ ]:
from sklearn.metrics import roc_curve, auc

for n in range(3):
    print('Модель ' + str(n+1) + ':')
    
    fpr, tpr, threshold = roc_curve(df['target'], df['pred' + str(n+1) + '_probs'])
    roc_auc = auc(fpr, tpr)

    plt.title('Receiver Operating Characteristic')
    plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
    plt.legend(loc = 'lower right')
    plt.plot([0, 1], [0, 1],'r--')
    plt.xlim([0, 1])
    plt.ylim([0, 1])
    plt.ylabel('True Positive Rate')
    plt.xlabel('False Positive Rate')
    plt.show()

### Выводы

**1. В данной задаче модель 2 выигрывает по всем метрикам. Однако не во всех задачах одна и та же модель будет лучшей относительно всех метрик!** 

**2. Качество решения задачи нельзя улучшить с помощью выбора метрики или настройки порога, но с помощью настройки порога можно добиться наилучшего результата в рамках поставленной задачи. Чтобы улучшить качество решения - необходимо изменять саму модель.**

#### Что ещё можно сделать

В python есть удобная функция classification_report, которая выводит значения нескольких метрик.

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(df['target'],df['pred1_classes']))

## Оценка качества в задачах многоклассовой классификации

### Precision, recall & confusion matrix для многоклассовой классификации

<img src="CM1.png">

<img src="CM2.png">

Для класса Cats:
    
    precision = доля правильно предсказанных Cats / все предсказанные Cats = 4/13
    
    recall = доля правильно предсказанных Cats / все истинные Cats = 4/6

In [ ]:
from sklearn import metrics

# Constants
C="Cat"
F="Fish"
H="Hen"

# True values
y_true = [C,C,C,C,C,C, F,F,F,F,F,F,F,F,F,F, H,H,H,H,H,H,H,H,H]
# Predicted values
y_pred = [C,C,C,C,H,F, C,C,C,C,C,C,H,H,F,F, C,C,C,H,H,H,H,H,H]

# Print the confusion matrix
print(metrics.confusion_matrix(y_true, y_pred))

# Print the precision and recall, among other metrics
print(metrics.classification_report(y_true, y_pred, digits=3))

### Macro avg (average)

***macro average (precision) = среднее арифметическое всех (precision)***

Пример:
***macro avg precision = (0.308+0.667+0.667)/3=0.547***

### Micro avg (average)

***micro average (precision) = precision, но вычисленная на всех данных вместе.***

Пример:
precision = TP/(TP+FP)

TP - все правильно предсказанные объекты: 
$$TP = 4 + 2 + 6 = 12$$

FP - например, все Cat, предсказанные как Fish и т.д.
$$FP = 6 + 3 + 1 + 0 + 1 + 2 = 13$$

Поэтому
***micro avg precision*** $ = 12/(12+13)=0.480$

### Cтатьи про метрики качества многоклассовой классификации:
    
1. https://towardsdatascience.com/multi-class-metrics-made-simple-part-i-precision-and-recall-9250280bddc2
        
2. https://towardsdatascience.com/multi-class-metrics-made-simple-part-ii-the-f1-score-ebe8b2c2ca1